In [1]:
import logging

from activitysim.core import tracing
from activitysim.core import config
from activitysim.core import pipeline
from activitysim.core import simulate
from activitysim.core import inject
from activitysim.core import logit

# from .util import expressions
# from .util import estimation

logger = logging.getLogger(__name__)

In [2]:
import pandas as pd
import numpy as np

In [21]:
## What it should be available at: 
households = pd.read_csv('../bay_area_base/data/households.csv')
persons = pd.read_csv('../bay_area_base/data/persons.csv').sample(100)

households = households.set_index('household_id')
persons = persons.set_index('person_id')

In [22]:
persons_merge = persons.merge(households, how = 'inner', left_on = 'household_id', right_index = True )

## telework_rates_csv
telework_option_rates = pd.DataFrame({'age_category':[0,0,0,1,1,1,2,2,2,3,3,3], 
                      'income_category':[0,1,2,0,1,2,0,1,2,0,1,2],
                      'rate':np.random.rand(12)})

telework_frequency_rates = pd.DataFrame({'0_days':[0.476],'1_days':[0.395],'2-3_days':[0.103],'4+_days':[0.026]})

telework_daily_rates = pd.DataFrame({'0_days':[0.0],'1_days':[0.2],'2-3_days':[0.5],'4+_days':[0.8]})

In [23]:
telework_option_rates

,age_category,income_category,rate
0,0,0,0.217527
1,0,1,0.609129
2,0,2,0.635800
3,1,0,0.513270
4,1,1,0.179913
5,1,2,0.312453
6,2,0,0.217232
7,2,1,0.243565
8,2,2,0.399214
9,3,0,0.465064


In [24]:
telework_frequency_rates

,0_days,1_days,2-3_days,4+_days
0,0.476,0.395,0.103,0.026


In [25]:
telework_daily_rates

,0_days,1_days,2-3_days,4+_days
0,0.0,0.2,0.5,0.8


## Telework as an Option

In [26]:
def find_index(array, value):
    """
    Returns the index where value is first found in array. If value is not found, returns NaN
    
    Parameters:
    ------------
    - array: n-dimensional array. Array of shape (n,m)
    - value: 1d-array of shape (m,)
    """
    
    not_found = True
    i = 0
    while not_found:
        try: 
            comparison = array[i,:] == value
        except IndexError:
            return np.nan
        if comparison.all():
            not_found = False
        else:
            i += 1
    return i

In [63]:
def find_rate(rates, array):
    "The df has the categories, and find the category combination in array and returns its index"

    index = []
    for cat in np.array(rates.drop(columns = 'rate')):
        i = find_index(category, cat)
        index.append(i)
    return index

In [53]:
choosers = persons_merge[persons_merge.ptype.isin([1,2])]

In [54]:
#persons_merge_annotate
age_category = pd.cut(choosers.age, [-np.inf,16,25,50,np.inf], labels = [0,1,2,3]).astype(int)
income_category = pd.cut(choosers.income, [-np.inf,75000,150000,np.inf], labels = [0,1,2]).astype(int)

choosers.insert(choosers.shape[1], 'age_category', age_category)
choosers.insert(choosers.shape[1], 'income_category', income_category)


In [55]:
# Processing
telework_rate_categories = ['age_category', 'income_category']
category, category_index = np.unique(choosers[telework_rate_categories].to_numpy(), axis=0, return_inverse=True)
choosers.insert(choosers.shape[1], 'telework_option_category', category_index)

In [66]:
corresponding_category = find_rate(telework_option_rates, category)

In [72]:
telework_option_rates.insert(0, 'telework_option_category', corresponding_category)

In [73]:
telework_option_rates

,telework_option_category,age_category,income_category,rate
0,NaN,0,0,0.217527
1,NaN,0,1,0.609129
2,NaN,0,2,0.635800
3,0.0,1,0,0.513270
4,1.0,1,1,0.179913
5,NaN,1,2,0.312453
6,2.0,2,0,0.217232
7,3.0,2,1,0.243565
8,4.0,2,2,0.399214
9,5.0,3,0,0.465064


In [75]:
rate_dict = telework_option_rates.set_index('telework_option_category')['rate']#.to_dict()

In [76]:
rate_dict

telework_option_category
NaN    0.217527
NaN    0.609129
NaN    0.635800
0.0    0.513270
1.0    0.179913
NaN    0.312453
2.0    0.217232
3.0    0.243565
4.0    0.399214
5.0    0.465064
6.0    0.888597
7.0    0.498130
Name: rate, dtype: float64

In [ ]:
persons_merge

In [81]:
# choosers['telework_probs'] = choosers['telework_option_category'].replace(rate_dict)
choosers.insert(choosers.shape[1], 'telework_option_probs', choosers['telework_option_category'].replace(rate_dict))

In [ ]:
1 - persons_merge['telework_probs']

In [ ]:
# Simulation
trace_label = 'telework'
probs = persons_merge[['telework_probs']]
probs.insert(0,'0', 1 - probs.telework_probs)

choices, rands = logit.make_choices(probs, trace_label=trace_label)

In [ ]:
# Simulation Result. Who telecommutes today. 
persons['telework'] = choices.reindex(persons.index).fillna(0).astype(bool)

In [ ]:
#Change ptype to 4 (non-workers which makes mandatory trips unaveilable)
new_ptype = persons['ptype'].mask(persons.telework, 4)

In [ ]:
@inject.step()
def telework(
        persons_merged, persons, households,
        skim_dict, skim_stack,
        chunk_size, trace_hh_id, locutor):
    
    """

    """

    trace_label = 'telework'
    model_settings_file_name = 'telework.yaml'

    choosers = persons_merged.to_frame()
    choosers = choosers[choosers.workplace_taz > -1] # Add filter that choosers are only workers
    
    logger.info("Running %s with %d persons", trace_label, len(choosers))

    model_settings = config.read_model_settings(model_settings_file_name)
#     constants = config.get_model_constants(model_settings)

    # - preprocessor
    
    add_rate_to_chooser()
    get_chooser_and_rates() #add probability of telecommuting and the compliment as well. 
    
#     preprocessor_settings = model_settings.get('preprocessor', None)
#     if preprocessor_settings:

#         locals_d = {}
#         if constants is not None:
#             locals_d.update(constants)

#         expressions.assign_columns(
#             df=choosers,
#             model_settings=preprocessor_settings,
#             locals_dict=locals_d,
#             trace_label=trace_label)

#     model_spec = simulate.read_model_spec(file_name=model_settings['SPEC'])
#     coefficients_df = simulate.read_model_coefficients(model_settings)
#     model_spec = simulate.eval_coefficients(model_spec, coefficients_df, estimator)

#     nest_spec = config.get_logit_model_settings(model_settings)


        ## FIX ME - This is way to complex function. I should just get the simulator because I have the
        ## probability ready (as the rate) 
#     choices = simulate.simple_simulate(
#         choosers=choosers,
#         spec=model_spec,
#         nest_spec=nest_spec,
#         locals_d=constants,
#         chunk_size=chunk_size,
#         trace_label=trace_label,
#         trace_choice_name='telework',
#         estimator=estimator)

    probs = #This hsoudl be a dataframe (row are users, columns are choices)
    choices, rands = logit.make_choices(probs, trace_label=trace_label)

    free_parking_alt = model_settings['FREE_PARKING_ALT']
    choices = (choices == free_parking_alt)

    persons = persons.to_frame()
    persons['telework'] = choices.reindex(persons.index).fillna(0).astype(bool)

    pipeline.replace_table("persons", persons)

    tracing.print_summary('telework', persons.telework, value_counts=True)

    if trace_hh_id:
        tracing.trace_df(persons,
                         label=trace_label,
                         warn_if_empty=True)